In [1]:
import ee
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import folium
import geemap

In [2]:
ee.Authenticate()
# Earth Engine API 초기화
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=wqw4___LNBJSyMNQjq_gzSOWA07HdGR8rfbaavHhaKw&tc=xJMTeBgocqSUspRamELG53RF7gGiPjEZMImTatymyGA&cc=Pt9axjat7CaGlQBzZMyZT_vyqHfoS2CiMajDNU9Qx4A

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXkB5SRCjkDPcWEmqKgy5g04phx_kjBMVVksHlwBK7Vr7bA3XJIeBQE

Successfully saved authorization token.


In [17]:
import ee
import geemap

# Earth Engine 초기화
ee.Initialize()

# 좌표 설정
coordinates = [
    [128.913574, 35.090979],
    [128.980522, 35.090979],
    [128.980522, 35.024936],
    [128.913574, 35.024936]
]

# 지정된 좌표로 지역을 생성
region = ee.Geometry.Polygon([coordinates])

# Dynamic World 데이터셋 불러오기
dynamicWorld = ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1")

# 2019년 1월 데이터 필터링
filteredImage = dynamicWorld.filterDate('2022-12-15', '2022-12-31').filterBounds(region)

# 필터링된 이미지의 첫 번째 이미지 가져오기
firstImage = filteredImage.first()

# 지도 생성 및 설정
Map = geemap.Map()
Map.centerObject(region, 10)
Map.addLayer(firstImage, {}, 'Dynamic World Image')

# 지도 출력
Map


Map(center=[35.05795764611553, 128.9470480000001], controls=(WidgetControl(options=['position', 'transparent_b…

In [ ]:
aoi = ee.Geometry.Polygon(
    [[[126.19653266055587, 36.853168924185226],
      [126.19653266055587, 36.83836240145186],
      [126.21259510596076, 36.83836240145186],
      [126.21259510596076, 36.853168924185226],
      [126.19653266055587, 36.853168924185226]]]
)

In [ ]:
start_date = '2021-01-01'
end_date = '2023-10-31'

In [ ]:
sentinel2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
            .filterBounds(aoi) \
            .filterDate(start_date, end_date) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)) \
            .first()

In [ ]:
def calculate_moisture(img):
    moisture = img.normalizedDifference(['B8A', 'B11'])
    return img.addBands(moisture.rename('moisture'))

def calculate_NDWI(img):
    NDWI = img.normalizedDifference(['B3', 'B8'])
    return img.addBands(NDWI.rename('NDWI'))

def water_bodies_index(img):
    moisture = img.select('moisture')
    NDWI = img.select('NDWI')
    water_bodies = NDWI.subtract(moisture).divide(NDWI.add(moisture))
    return img.addBands(water_bodies.rename('water_bodies'))

def calculate_water_plants(img):
    water_plants = img.normalizedDifference(['B5', 'B4'])
    NIR2 = img.expression(
        'B4 + (B11 - B4) * ((832.8 - 664.6) / (1613.7 - 664.6))',
        {
            'B4': img.select('B4'),
            'B11': img.select('B11')
        }
    )
    FAI = img.select('B8').subtract(NIR2)

    # # Calculate underwater vegetation index
    # UVI = img.normalizedDifference(['B3', 'B5'])

    # return img.addBands(water_plants.rename('water_plants')).addBands(FAI.rename('FAI')).addBands(UVI.rename('UVI'))
    return img.addBands(water_plants.rename('water_plants')).addBands(FAI.rename('FAI'))

def calculate_wevi(img):
    # WEVI 계산: (B3 - B4) / (B3 + B4 - B2)
    green = img.select('B3')
    red = img.select('B4')
    blue = img.select('B2')

    wevi = green.subtract(red).divide(green.add(red).subtract(blue))
    return img.addBands(wevi.rename('WEVI'))

def calculate_wavi(img, alpha=1, beta=0.1):
    # WAVI 계산: (B8 - B4) / (B8 + B4 - α * B2 + β)
    NIR = img.select('B8')  # Near-Infrared
    RED = img.select('B4')  # Red
    BLUE = img.select('B2')  # Blue

    wavi = NIR.subtract(RED).divide(NIR.add(RED).subtract(BLUE.multiply(alpha)).add(beta)).rename('WAVI')
    return img.addBands(wavi)

def calculate_ndvi(img):
    # NDVI 계산: (B8 - B4) / (B8 + B4)
    NIR = img.select('B8')  # Near-Infrared
    RED = img.select('B4')  # Red

    ndvi = NIR.subtract(RED).divide(NIR.add(RED)).rename('NDVI')
    return img.addBands(ndvi)

# def cloud_mask(img):
#     bRatio = img.normalizedDifference(['B3', 'B4'])
#     NDGR = img.normalizedDifference(['B3', 'B4'])
#     mask = bRatio.gt(1).Or(bRatio.gt(0).And(NDGR.gt(0)))
#     return img.updateMask(mask.Not())

def process_image(img):
    img = calculate_moisture(img)
    img = calculate_NDWI(img)
    img = water_bodies_index(img)
    img = calculate_water_plants(img)
    # img = cloud_mask(img)  # 구름 마스킹은 현재 주석 처리되어 있습니다.
    return img
# 신두리사구의 좌표를 사용하여 영역을 정의
# 신두리사구의 좌표를 사용하여 영역을 정의
sinduri_location = ee.Geometry.Polygon([
    [128.913574, 35.024936],
    [128.913574, 35.090979],
    [128.980522, 35.090979],
    [128.980522, 35.024936],
    [128.913574, 35.024936]
])
# 예제 이미지에 함수 적용 (이미지 ID는 적절하게 변경해야 함)
example_image_collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')\
    .filterDate('2022-10-25', '2023-10-30')\
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))\
    .filterBounds(sinduri_location)

# ImageCollection에서 첫 번째 이미지 선택
example_image = example_image_collection.first()  # 첫 번째 이미지를 선택

processed_image = process_image(example_image)
# 지도 생성 및 시각화
Map = geemap.Map()
Map.centerObject(sinduri_location, 14)  # 신두리사구를 중심으로 설정

# 육지를 마스킹하기 위한 조건 설정
# water_mask = processed_image.select('UVI').gt(0.5)  # 예시로 0.5보다 큰 값인 픽셀을 수상으로 간주

# 처리된 이미지를 지도에 추가 (수중식물 지수 사용) - 수상만 표시
uvi_params = {
    'bands': ['FAI'],  # UVI 밴드만 사용
    'min': -1000,  # 수중식물 지수의 최소값
    'max': 1000,   # 수중식물 지수의 최대값
    'palette': ['purple', 'blue', 'green', 'yellow', 'red']  # 색상 팔레트 설정
}
Map.addLayer(processed_image, uvi_params, 'Processed Image (FAI)')

# # 다른 밴드도 추가할 경우 (False Color Composite 사용) - 육지 마스킹 적용
# false_color_params = {
#     'bands': ['B8', 'B4', 'B3'],  # B8, B4, B3 밴드를 사용하여 가짜색 합성
#     'min': 0,
#     'max': 3000
# }
# Map.addLayer(processed_image.updateMask(water_mask), false_color_params, 'Processed Image (False Color - Water Only)')

# 지도를 화면에 표시
Map

Map(center=[35.05795764611553, 128.9470480000001], controls=(WidgetControl(options=['position', 'transparent_b…

In [ ]:
acquisition_time = ee.Date(example_image.get('system:time_start'))

# 출력
print('Acquisition time:', acquisition_time.format('YYYY-MM-dd HH:mm:ss').getInfo())

Acquisition time: 2022-11-05 02:17:53


In [ ]:
import ee
import geemap

ee.Initialize()


# 예제 이미지에 함수 적용 (이미지 ID는 적절하게 변경해야 함)
example_image_collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')\
    .filterDate('2022-09-25', '2022-10-30')\
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))\
    .filterBounds(sinduri_location)

# ImageCollection에서 첫 번째 이미지 선택
example_image = example_image_collection.first()  # 첫 번째 이미지를 선택

# 기본 광학영상 시각화 매개변수
rgb_params = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 3000
}

# 지도 생성 및 기본 광학영상 추가
Map = geemap.Map()
Map.centerObject(sinduri_location, 14)  # 신두리사구를 중심으로 설정
Map.addLayer(example_image, rgb_params, 'Optical Image')

# 지도를 화면에 표시
Map


Map(center=[35.05795764611553, 128.9470480000001], controls=(WidgetControl(options=['position', 'transparent_b…

In [ ]:
import ee
import folium

ee.Initialize()

# 지정된 영역 정의
geometry = ee.Geometry.Polygon([
    [128.913574, 35.024936],
    [128.913574, 35.090979],
    [128.980522, 35.090979],
    [128.980522, 35.024936],
    [128.913574, 35.024936]
])

# Sentinel-1 이미지 컬렉션 필터링
sentinel1 = ee.ImageCollection('COPERNICUS/S1_GRD')\
    .filterBounds(geometry)\
    .filterDate('2022-09-25', '2022-10-30')\
    .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))\
    .first() # 첫 번째 이미지 선택

# RVI 계산
rvi = sentinel1.expression(
    '4 * sqrt(VH / VV)', {
        'VH': sentinel1.select('VH'),
        'VV': sentinel1.select('VV')
    })

# 시각화를 위한 색상 및 그라데이션 설정
rviVisualization = {
    'min': 0.0,
    'max': 10.0,
    'palette': ['blue', 'green', 'yellow', 'red']
}

# Folium 지도 생성
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Folium에 이 메서드를 추가
folium.Map.add_ee_layer = add_ee_layer

# 지도 생성 및 RVI 레이어 추가
map = folium.Map(location=[35.057962, 128.947022], zoom_start=12)
map.add_ee_layer(rvi, rviVisualization, 'RVI')

# 지도에 레이어 컨트롤 추가
map.add_child(folium.LayerControl())

# 지도 표시
map


In [ ]:
stats = rvi.reduceRegion(
    reducer=ee.Reducer.minMax().combine(
        reducer2=ee.Reducer.mean(),
        sharedInputs=True
    ),
    geometry=geometry,
    scale=10, # 적절한 scale 값 설정 필요
    maxPixels=1e9
)

# 결과 출력
print(stats.getInfo())

{'constant_max': 2006.5872286104916, 'constant_mean': 5.58630139866211, 'constant_min': 0.1169169726803609}


In [ ]:
# 향상된 백분위수 계산
percentiles = rvi.reduceRegion(
    reducer=ee.Reducer.percentile([5, 10, 25, 75, 90, 95]), # 다양한 백분위수
    geometry=geometry,
    scale=10, # 적절한 scale 값 설정 필요
    maxPixels=1e9
)

# 평균 및 표준편차 계산
stats = rvi.reduceRegion(
    reducer=ee.Reducer.mean().combine(
        reducer2=ee.Reducer.stdDev(), sharedInputs=True
    ),
    geometry=geometry,
    scale=10,
    maxPixels=1e9
)

# 백분위수 결과 및 평균/표준편차 결과 출력
print('Percentiles:', percentiles.getInfo())
print('Mean and StdDev:', stats.getInfo())

Percentiles: {'constant_p10': 5.186907172353848, 'constant_p25': 5.186907172353848, 'constant_p5': 5.186907172353848, 'constant_p75': 5.186907172353848, 'constant_p90': 5.186907172353848, 'constant_p95': 5.186907172353848}
Mean and StdDev: {'constant_mean': 5.58630139866211, 'constant_stdDev': 5.4536539023922845}


In [ ]:
import ee

ee.Initialize()

# 지정된 영역 정의
geometry = ee.Geometry.Polygon([
    [128.913574, 35.024936],
    [128.913574, 35.090979],
    [128.980522, 35.090979],
    [128.980522, 35.024936],
    [128.913574, 35.024936]
])

# Sentinel-1 이미지 컬렉션 필터링
sentinel1 = ee.ImageCollection('COPERNICUS/S1_GRD')\
    .filterBounds(geometry)\
    .filterDate('2022-09-25', '2022-10-30')\
    .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))\
    .first() # 첫 번째 이미지 선택

# RVI 계산
rvi = sentinel1.expression(
    '4 * sqrt(VH / VV)', {
        'VH': sentinel1.select('VH'),
        'VV': sentinel1.select('VV')
    })

# 히스토그램 계산
histogram = rvi.reduceRegion(
    reducer=ee.Reducer.fixedHistogram(0, 1, 100),
    geometry=geometry,
    scale=10,
    maxPixels=1e9
)

# 백분위수 계산
percentiles = rvi.reduceRegion(
    reducer=ee.Reducer.percentile([5, 95]), # 하위 5% 및 상위 95% 백분위수
    geometry=geometry,
    scale=10,
    maxPixels=1e9
)

# 평균 및 표준편차 계산
stats = rvi.reduceRegion(
    reducer=ee.Reducer.mean().combine(
        reducer2=ee.Reducer.stdDev(), sharedInputs=True
    ),
    geometry=geometry,
    scale=10,
    maxPixels=1e9
)

# 결과 출력
print('Histogram:', histogram.getInfo())
print('Percentiles:', percentiles.getInfo())
print('Mean and StdDev:', stats.getInfo())


Histogram: {'constant': [[0, 0], [0.01, 0], [0.02, 0], [0.03, 0], [0.04, 0], [0.05, 0], [0.06, 0], [0.07, 0], [0.08, 0], [0.09, 0], [0.1, 0], [0.11, 1], [0.12, 1], [0.13, 3], [0.14, 1], [0.15, 0], [0.16, 2], [0.17, 1], [0.18, 2], [0.19, 2], [0.2, 0], [0.21, 0], [0.22, 0], [0.23, 0], [0.24, 0], [0.25, 0], [0.26, 0], [0.27, 1], [0.28, 2], [0.29, 0], [0.3, 0], [0.31, 0], [0.32, 2], [0.33, 0], [0.34, 0], [0.35000000000000003, 0], [0.36, 0], [0.37, 0], [0.38, 0], [0.39, 1], [0.4, 1], [0.41000000000000003, 0], [0.42, 4], [0.43, 2], [0.44, 3], [0.45, 0], [0.46, 0], [0.47000000000000003, 0], [0.48, 1], [0.49, 1], [0.5, 3], [0.51, 2], [0.52, 0], [0.53, 1], [0.54, 1], [0.55, 1], [0.56, 0], [0.5700000000000001, 0], [0.58, 0], [0.59, 2], [0.6, 1], [0.61, 1], [0.62, 4], [0.63, 1], [0.64, 0], [0.65, 1], [0.66, 1], [0.67, 0], [0.68, 0], [0.6900000000000001, 2], [0.7000000000000001, 0], [0.71, 2], [0.72, 2], [0.73, 2], [0.74, 1], [0.75, 2], [0.76, 1], [0.77, 1], [0.78, 2], [0.79, 4], [0.8, 1], [0.81, 

In [ ]:
import ee
import folium

ee.Initialize()

# 지정된 영역 정의
geometry = ee.Geometry.Polygon([
    [128.913574, 35.024936],
    [128.913574, 35.090979],
    [128.980522, 35.090979],
    [128.980522, 35.024936],
    [128.913574, 35.024936]
])

# Sentinel-1 이미지 컬렉션 필터링: VV와 VH 폴라리제이션 채널이 모두 있는 이미지만 선택
sentinel1 = ee.ImageCollection('COPERNICUS/S1_GRD')\
    .filterBounds(geometry)\
    .filterDate('2022-09-25', '2022-10-30')\
    .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))\
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))\
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
    .first() # 첫 번째 이미지 선택

# RVI 계산
rvi = sentinel1.expression(
    '4 * sqrt(VH / VV)', {
        'VH': sentinel1.select('VH'),
        'VV': sentinel1.select('VV')
    })

# 시각화를 위한 색상 및 그라데이션 설정
rviVisualization = {
    'min': 0,
    'max': 10.0,
    'palette': ['blue', 'green', 'yellow', 'red']
}

# Folium 지도 생성
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Folium에 이 메서드를 추가
folium.Map.add_ee_layer = add_ee_layer

# 지도 생성 및 RVI 레이어 추가
map = folium.Map(location=[35.057962, 128.947022], zoom_start=12)
map.add_ee_layer(rvi, rviVisualization, 'RVI')

# 지도에 레이어 컨트롤 추가
map.add_child(folium.LayerControl())

# 지도 표시
map


Acquisition time: 2022-10-01 02:18:03


In [ ]:
import ee
import folium

# Google Earth Engine 초기화
ee.Initialize()

# 관심 지역 설정 (예: 울숙도)
area_of_interest = ee.Geometry.Polygon([
    [128.913574, 35.024936],
    [128.913574, 35.090979],
    [128.980522, 35.090979],
    [128.980522, 35.024936],
    [128.913574, 35.024936]
])

# Sentinel-2 이미지 컬렉션 필터링
sentinel2 = ee.ImageCollection('COPERNICUS/S2')\
    .filterBounds(area_of_interest)\
    .filterDate('2023-01-01', '2023-03-30')\
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) # 구름이 적은 이미지 선택

# NDVI 계산 함수
def calculate_ndvi(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

# NDVI 계산 적용
ndvi_images = sentinel2.map(calculate_ndvi)

# NDVI의 평균 이미지 선택
ndvi_avg = ndvi_images.mean()

# 시각화 매개변수 설정
ndvi_params = {'bands': ['NDVI'], 'min': -0.14, 'max': 0.1, 'palette': ['blue', 'white', 'green']}

# Folium 지도 생성
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Folium에 이 메서드를 추가
folium.Map.add_ee_layer = add_ee_layer

# 지도 생성 및 NDVI 레이어 추가
map = folium.Map(location=[35.090979, 128.913574], zoom_start=12)
map.add_ee_layer(ndvi_avg, ndvi_params, 'NDVI')

# 지도에 레이어 컨트롤 추가
map.add_child(folium.LayerControl())

# 지도 표시
map


In [ ]:
stats = ndvi_avg.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=area_of_interest,
    scale=10, # 적절한 scale 값 설정 필요
    maxPixels=1e9
)

# 결과 출력
stats_dict = stats.getInfo()
print('NDVI 최소값:', stats_dict['NDVI_min'])
print('NDVI 최대값:', stats_dict['NDVI_max'])

NDVI 최소값: -0.1477426141500473
NDVI 최대값: 0.46002936363220215


랜드셋

In [ ]:
import ee
import folium

# Google Earth Engine 초기화
ee.Initialize()

# 관심 지역 설정
aoi = ee.Geometry.Polygon([
    [128.913574, 35.024936],
    [128.913574, 35.090979],
    [128.980522, 35.090979],
    [128.980522, 35.024936],
    [128.913574, 35.024936]
])

# Landsat 8 이미지 컬렉션 필터링
landsat = ee.ImageCollection('LANDSAT/LC08/C01/T1')\
    .filterBounds(aoi)\
    .filterDate('2020-01-01', '2022-03-30')\
    .first()  # 첫 번째 이미지 선택

# 시각화 매개변수 설정
vis_params = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}

# Folium 지도 생성
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Folium에 이 메서드를 추가
folium.Map.add_ee_layer = add_ee_layer

# 지도 생성 및 Landsat 이미지 레이어 추가
map = folium.Map(location=[35.057962, 128.947022], zoom_start=12)
map.add_ee_layer(landsat, vis_params, 'Landsat 8 Image')

# 지도에 레이어 컨트롤 추가
map.add_child(folium.LayerControl())

# 지도 표시
map


In [ ]:
aoi = ee.Geometry.Polygon([
    [128.913574, 35.024936],
    [128.913574, 35.090979],
    [128.980522, 35.090979],
    [128.980522, 35.024936]
])

# Landsat 8 이미지 컬렉션 필터링
landsat8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')\
    .filterBounds(aoi)\
    .filterDate('2021-10-15', '2022-01-31')\
    .first()  # 첫 번째 이미지 선택

# 시각화 매개변수 설정
vis_params = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}

# Folium 지도 생성 함수
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Folium에 이 메서드를 추가
folium.Map.add_ee_layer = add_ee_layer

# 지도 생성 및 Landsat 8 이미지 레이어 추가
map = folium.Map(location=[35.057962, 128.947022], zoom_start=12)
map.add_ee_layer(landsat8, vis_params, 'Landsat 8 Image')

# 지도에 레이어 컨트롤 추가
map.add_child(folium.LayerControl())

# 지도 표시
map